In [12]:
import pandas as pd 
import numpy as np

#load the faculty's information
df_f = pd.read_csv("faculty.csv",delimiter=',')
df_f

,Name,Surname
0,Athena,Stassopoulou
1,Charalambos,Christou
2,Constandinos,Mavromoustakis
3,Demetris,Trihinas
4,Dmitry,Apraksin
5,George,Chailos
6,George,Portides
7,Harald,Gjermundrod
8,Ioanna,Dionysiou
9,Ioannis,Katakis


In [13]:
#Asking the user to enter a professor's name 
name=input("Enter the professor's name: ")

Enter the professor's name: Constantinos Mavromoustakis


In [14]:
#crating a list of the professors' full names 
#and appending them into the list and later on adding it to the dataframe
name_text=[]
for index, row in df_f.iterrows():
    name_text.append(row['Name'].strip()+' '+row['Surname'].strip())
    
df_f['full_name']=name_text
df_f['score']=np.nan
df_f

,Name,Surname,full_name,score
0,Athena,Stassopoulou,Athena Stassopoulou,NaN
1,Charalambos,Christou,Charalambos Christou,NaN
2,Constandinos,Mavromoustakis,Constandinos Mavromoustakis,NaN
3,Demetris,Trihinas,Demetris Trihinas,NaN
4,Dmitry,Apraksin,Dmitry Apraksin,NaN
5,George,Chailos,George Chailos,NaN
6,George,Portides,George Portides,NaN
7,Harald,Gjermundrod,Harald Gjermundrod,NaN
8,Ioanna,Dionysiou,Ioanna Dionysiou,NaN
9,Ioannis,Katakis,Ioannis Katakis,NaN


In [15]:
#in case of wrong spelling we estimate the closest professor's name by the library 'similarity'
#which calculates an estimates the closest number for every professor's name
from similarity.jarowinkler import JaroWinkler

jarowinkler = JaroWinkler()
score_list=[]
for index, row in df_f.iterrows():
    score_list.append(jarowinkler.similarity(row['full_name'], name))

#storing all the name scores from the above calculations
df_f['score']=score_list
df_f

,Name,Surname,full_name,score
0,Athena,Stassopoulou,Athena Stassopoulou,0.553119
1,Charalambos,Christou,Charalambos Christou,0.582289
2,Constandinos,Mavromoustakis,Constandinos Mavromoustakis,0.981710
3,Demetris,Trihinas,Demetris Trihinas,0.472767
4,Dmitry,Apraksin,Dmitry Apraksin,0.443210
5,George,Chailos,George Chailos,0.491182
6,George,Portides,George Portides,0.432451
7,Harald,Gjermundrod,Harald Gjermundrod,0.455247
8,Ioanna,Dionysiou,Ioanna Dionysiou,0.636455
9,Ioannis,Katakis,Ioannis Katakis,0.654511


In [16]:
#find the highest score in order to provide to the user the closest professor's name to his/her input
max_index=df_f['score'].idxmax()

In [17]:
import urllib3
urllib3.disable_warnings()
from bs4 import BeautifulSoup
import re

#load the htlm code from the official University's website to extract the professor's CV that the user has asked

http = urllib3.PoolManager()
#passing the name and surname of the professor in the url
url=f'https://www.unic.ac.cy/{df_f.iloc[max_index,1].strip()}-{df_f.iloc[max_index,0].strip()}/'
response = http.request('GET', url)
#cleaning the htlm code to get the plain text
soup = BeautifulSoup(response.data)
soup = soup.find('div', attrs={'class':'fusion-text'})

nps= ''.join(str (soup))
nps=re.sub(r':','',nps)
nps=re.sub(r'%','',nps)
nps=re.sub(r'<[\w\s\"\./=-]*>','',nps)

print (nps)

Constandinos X. Mavromoustakis (male, Prof., PhD. since 2006) is currently a Professor at the Department of Computer Science at the University of Nicosia, Cyprus. He received a five-year dipl.Eng (BSc, BEng, Meng/KISATS approved/accredited) in Electronic and Computer Engineering from Technical University of Crete, Greece, MSc in Telecommunications from University College of London, UK, and his PhD from the department of Informatics at Aristotle University of Thessaloniki, Greece. Professor Mavromoustakis is leading the Mobile Systems Lab. (MOSys Lab., http//www.mosys.unic.ac.cy/) at the Department of Computer Science at the University of Nicosia and he is an active member (vice-chair) of IEEE/ R8 regional Cyprus section since Jan. 2016, and since May 2009 he serves as the Chair of C16 Computer Society Chapter of the Cyprus IEEE section. Prof. Mavromoustakis has a dense research work outcome in Mobile and Wearable computing systems and the Internet-of-Things (IoT), consisting of numerou

In [18]:
#load the csv file with the supported langauges
df = pd.read_csv("lang.csv",delimiter=',')

#creating a dictionary to match each langauge with its code
d=dict()
i=0
while i<len(df):
    d[df.iloc[i,0].lower()]=df.iloc[i,1]
    i+=1

# Translation

In [19]:
#ask the user the target language
lang=input("Enter the language you want your text to be translated: ")
code=d.get(lang.lower())
print()

#translate the CV into the asked language
from py_translator import Translator

#check if the language exist in the dictionary
if code!=None:
    s = Translator().translate(text=nps, dest=code).text
    print(s)
else: 
    print('Not valid language.')

Enter the language you want your text to be translated: greek

Ο Κωνσταντίνος Χ. Μαυρομουστάκης (άνδρας, Καθηγητής, από το 2006) είναι Καθηγητής στο Τμήμα Επιστήμης Υπολογιστών του Πανεπιστημίου Λευκωσίας της Κύπρου. Έλαβε πενταετή πτυχίο (BSc, BEng, Meng / KISATS εγκεκριμένο / αναγνωρισμένο) στην Ηλεκτρονική και Μηχανική Υπολογιστών από το Πολυτεχνείο Κρήτης, MSc στον τομέα των Τηλεπικοινωνιών από το Πανεπιστήμιο του Λονδίνου στο Ηνωμένο Βασίλειο και το Διδακτορικό του Τμήμα Πληροφορικής του Αριστοτελείου Πανεπιστημίου Θεσσαλονίκης. Ο καθηγητής Μαυρομουστάκης ηγείται του εργαστηρίου Mobile Systems. (Εργαστήριο MOSys, http // www.mosys.unic.ac.cy /) στο Τμήμα Επιστήμης Υπολογιστών του Πανεπιστημίου Λευκωσίας και είναι ενεργό μέλος (αντιπρόεδρος) του περιφερειακού τμήματος Κύπρου IEEE / R8 από τον Ιανουάριο Το 2016 και από το Μάιο του 2009 ως Πρόεδρος του Τμήματος Κοινωνίας της Πληροφορίας C16 του τμήματος IEEE Κύπρου. Ο καθηγητής Μαυρομουστάκης έχει ένα πυκνό ερευνητικό αποτέλεσμα στα 

# Identification


In [20]:
#randomly select a language to translate the CV (from the list)

from random import *
items = ['en','fr','de','it','es','fi']
x = sample(items,  1)   # Pick a random item from the list

print (x[0])

nps = Translator().translate(text=nps, dest=x[0]).text
nps

de


'Constandinos X. Mavromoustakis (männlich, seit 2006) ist Professor an der Fakultät für Informatik an der Universität von Nikosia, Zypern. Er erhielt ein fünfjähriges Diplom in Elektrotechnik und Informatik von der Technischen Universität von Kreta (Griechenland), MSc in Telekommunikation vom University College of London (Großbritannien) und promovierte an der University of London Abteilung für Informatik an der Aristoteles-Universität von Thessaloniki, Griechenland. Professor Mavromoustakis leitet das Mobile Systems Lab. (MOSys Lab., Http // www.mosys.unic.ac.cy /) an der Fakultät für Informatik der Universität von Nikosia und ist seit Jan. Im aktiven Bereich (stellvertretender Vorsitzender) der regionalen Sektion IEEE / R8 in Zypern 2016 und seit Mai 2009 ist er Vorsitzender des Abschnitts C16 Computer Society der zyprischen Sektion IEEE. Prof. Mavromoustakis verfügt über umfangreiche Forschungsergebnisse in den Bereichen Mobile und Wearable-Computersysteme und Internet of Things (Io

In [21]:
#remove punctuations and numbers

nps=re.sub(r'\d','',nps)
nps=re.sub('[^A-Za-z0-9]+', ' ', nps)
nps=nps.lower()

print (nps)


constandinos x mavromoustakis m nnlich seit ist professor an der fakult t f r informatik an der universit t von nikosia zypern er erhielt ein f nfj hriges diplom in elektrotechnik und informatik von der technischen universit t von kreta griechenland msc in telekommunikation vom university college of london gro britannien und promovierte an der university of london abteilung f r informatik an der aristoteles universit t von thessaloniki griechenland professor mavromoustakis leitet das mobile systems lab mosys lab http www mosys unic ac cy an der fakult t f r informatik der universit t von nikosia und ist seit jan im aktiven bereich stellvertretender vorsitzender der regionalen sektion ieee r in zypern und seit mai ist er vorsitzender des abschnitts c computer society der zyprischen sektion ieee prof mavromoustakis verf gt ber umfangreiche forschungsergebnisse in den bereichen mobile und wearable computersysteme und internet of things iot die aus zahlreichen referierten publikationen gt 

In [22]:
#add languages profiles in a list
categories_list=['N_Gram_English.txt','N_Gram_French.txt','N_Gram_German.txt','N_Gram_Spannish.txt','N_Gram_Italian.txt','N_Gram_Finnish.txt']
new_list_languages=[]
temp_list=[]
for i in range(len(categories_list)):
    temp_list=[]
    f=open(categories_list[i], "r")
    contents =f.readlines()
    for x in contents:
        temp_list.append(x.rstrip('\n'))
    f.close()
    new_list_languages.append(temp_list)
    


In [23]:
#tokenization
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer("[a-zA-Z'`éèî]+")
token=list(tokenizer.tokenize(nps))
print(token)

['constandinos', 'x', 'mavromoustakis', 'm', 'nnlich', 'seit', 'ist', 'professor', 'an', 'der', 'fakult', 't', 'f', 'r', 'informatik', 'an', 'der', 'universit', 't', 'von', 'nikosia', 'zypern', 'er', 'erhielt', 'ein', 'f', 'nfj', 'hriges', 'diplom', 'in', 'elektrotechnik', 'und', 'informatik', 'von', 'der', 'technischen', 'universit', 't', 'von', 'kreta', 'griechenland', 'msc', 'in', 'telekommunikation', 'vom', 'university', 'college', 'of', 'london', 'gro', 'britannien', 'und', 'promovierte', 'an', 'der', 'university', 'of', 'london', 'abteilung', 'f', 'r', 'informatik', 'an', 'der', 'aristoteles', 'universit', 't', 'von', 'thessaloniki', 'griechenland', 'professor', 'mavromoustakis', 'leitet', 'das', 'mobile', 'systems', 'lab', 'mosys', 'lab', 'http', 'www', 'mosys', 'unic', 'ac', 'cy', 'an', 'der', 'fakult', 't', 'f', 'r', 'informatik', 'der', 'universit', 't', 'von', 'nikosia', 'und', 'ist', 'seit', 'jan', 'im', 'aktiven', 'bereich', 'stellvertretender', 'vorsitzender', 'der', 'reg

In [24]:
#ngram generator 
ande=[]
neword=''
for i in range(len(token)):
    for j in range(2,6):
        num=0
        num_of_letter=0
        new_letter=0
        for k in range(len(token[i])+1):
            if k-1<0:
                neword+='_'
                for p in range(j-1):
                    if(j-1-p<=len(token[i])):
                        neword+= token[i][new_letter]
                        new_letter+=1
                    else:
                        neword+='_'
                new_letter = 0
            elif num < len(token[i]) - j + 1 & j <= len(token[i]):
                for n in range(j):
                    neword += token[i][new_letter]
                    new_letter+=1
                new_letter = new_letter - (j - 1);
                num+=1;
            else:
                for so in range (j):
                    if k+so <=len(token[i]):
                        neword+=token[i][k-1+so]
                        
                    else:
                        neword+='_'
                
                        
            ande.append(neword)
            neword = ""
print(ande)


['_c', 'co', 'on', 'ns', 'st', 'ta', 'an', 'nd', 'di', 'in', 'no', 'os', 's_', '_co', 'con', 'ons', 'nst', 'sta', 'tan', 'and', 'ndi', 'din', 'ino', 'nos', 'os_', 's__', '_con', 'cons', 'onst', 'nsta', 'stan', 'tand', 'andi', 'ndin', 'dino', 'inos', 'nos_', 'os__', 's___', '_cons', 'const', 'onsta', 'nstan', 'stand', 'tandi', 'andin', 'ndino', 'dinos', 'inos_', 'nos__', 'os___', 's____', '_x', 'x_', '__x', 'x__', '___x', 'x___', '____x', 'x____', '_m', 'ma', 'av', 'vr', 'ro', 'om', 'mo', 'ou', 'us', 'st', 'ta', 'ak', 'ki', 'is', 's_', '_ma', 'mav', 'avr', 'vro', 'rom', 'omo', 'mou', 'ous', 'ust', 'sta', 'tak', 'aki', 'kis', 'is_', 's__', '_mav', 'mavr', 'avro', 'vrom', 'romo', 'omou', 'mous', 'oust', 'usta', 'stak', 'taki', 'akis', 'kis_', 'is__', 's___', '_mavr', 'mavro', 'avrom', 'vromo', 'romou', 'omous', 'moust', 'ousta', 'ustak', 'staki', 'takis', 'akis_', 'kis__', 'is___', 's____', '_m', 'm_', '__m', 'm__', '___m', 'm___', '____m', 'm____', '_n', 'nn', 'nl', 'li', 'ic', 'ch', 'h_

In [25]:
#hashing
ngrams_statistics = {}
for ngram in ande:
    if ngram in ngrams_statistics:
        ngram_occurrences = ngrams_statistics[ngram]
        ngrams_statistics.update({ngram:ngram_occurrences+1})
    else:
        ngrams_statistics.update({ngram:1})
        
ngrams_statistics


{'_c': 13,
 'co': 10,
 'on': 20,
 'ns': 5,
 'st': 16,
 'ta': 8,
 'an': 14,
 'nd': 21,
 'di': 4,
 'in': 19,
 'no': 2,
 'os': 5,
 's_': 18,
 '_co': 10,
 'con': 1,
 'ons': 3,
 'nst': 1,
 'sta': 6,
 'tan': 4,
 'and': 5,
 'ndi': 1,
 'din': 1,
 'ino': 1,
 'nos': 1,
 'os_': 1,
 's__': 18,
 '_con': 1,
 'cons': 1,
 'onst': 1,
 'nsta': 1,
 'stan': 3,
 'tand': 3,
 'andi': 1,
 'ndin': 1,
 'dino': 1,
 'inos': 1,
 'nos_': 1,
 'os__': 1,
 's___': 18,
 '_cons': 1,
 'const': 1,
 'onsta': 1,
 'nstan': 1,
 'stand': 3,
 'tandi': 1,
 'andin': 1,
 'ndino': 1,
 'dinos': 1,
 'inos_': 1,
 'nos__': 1,
 'os___': 1,
 's____': 18,
 '_x': 1,
 'x_': 1,
 '__x': 1,
 'x__': 1,
 '___x': 1,
 'x___': 1,
 '____x': 1,
 'x____': 1,
 '_m': 12,
 'ma': 8,
 'av': 3,
 'vr': 3,
 'ro': 10,
 'om': 15,
 'mo': 8,
 'ou': 3,
 'us': 5,
 'ak': 6,
 'ki': 4,
 'is': 11,
 '_ma': 4,
 'mav': 3,
 'avr': 3,
 'vro': 3,
 'rom': 4,
 'omo': 4,
 'mou': 3,
 'ous': 3,
 'ust': 4,
 'tak': 3,
 'aki': 3,
 'kis': 3,
 'is_': 3,
 '_mav': 3,
 'mavr': 3,
 'avro'

In [26]:
#sorting
import operator
ngrams_statistics_sorted = sorted(ngrams_statistics.items(),key=operator.itemgetter(1),reverse=True)[0:300]

sup_list=[]
for i in range(len(ngrams_statistics_sorted)):
    sup_list.append(ngrams_statistics_sorted[i][0])
print(sup_list)


['er', 'n_', 'n__', 'n___', 'n____', 'r_', 'r__', 'r___', 'r____', 'er_', 'er__', 'er___', 't_', 't__', 't___', 't____', '_i', 'un', 'te', 'en', 'nd', 'on', 'ie', 'in', 'ch', 'de', 's_', 's__', 's___', 's____', '_d', '_u', 'ni', 'e_', 'e__', 'e___', 'e____', 'it', '_un', 'st', 'en_', 'en__', 'en___', 'om', 'ei', '_de', 'der', 'der_', 'der__', 'an', 'd_', 'd__', 'd___', 'd____', 're', '_c', '_s', 'ti', '_e', '_m', 'or', '_a', '_der', '__der', 'at', '_in', 'nd_', 'nd__', 'nd___', 'is', 'ic', '_t', 'rs', '_v', 'on_', 'on__', 'on___', 'he', 'io', 'ee', 'co', '_co', 'ro', 'f_', 'f__', 'f___', 'f____', '_r', 'ik', 'ati', 've', 'si', 'uni', 'le', 'und', '_und', 'und_', '__und', 'und__', 'ver', 'vo', '_vo', 'ri', 'che', 'ion', 'ta', 'ma', 'mo', 'li', 'ich', 'se', 'es', '_f', 'rsi', 'sit', 'rsit', 'el', 'sc', 'c_', 'c__', 'c___', 'c____', 'tio', 'tion', '_l', 'it_', 'it__', 'it___', 'pr', 'an_', 'an__', 'an___', 'iv', 'ive', 'ers', '_uni', 'hen', '_b', 'com', '_com', 'sta', 'ak', '_p', 'of', '_

In [27]:
import math
lang=['English','French','German','Spannish','Italian','Finnish']


languages_points = {0 : 0, 1: 0, 2 : 0, 3 : 0, 4 : 0, 5 : 0}

#languages new_list_languages
#new language sup_list

for i in range (len(new_list_languages)):
    counter=0
    for j in range(len(sup_list)):
        for n in range(len(new_list_languages[i])):
            if sup_list[j]==new_list_languages[i][n]:
                counter+= math.sqrt((j-n)**2)
                break

        
        if n==299:#len(new_list_languages)-1:
            counter+=len(sup_list)
        
    languages_points[i]=counter

ind=min(languages_points, key=languages_points.get)


print('The origin lanuage of the new text is : '+ lang[ind]+' with '+str(languages_points[ind])+' points')


The origin lanuage of the new text is : German with 53405.0 points
